In [1]:
!nvidia-smi

Fri Dec 22 18:56:42 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/MTminiproject/densray-debiasing-publish/
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/MTminiproject/densray-debiasing-publish
 conceptor_bert.py		      professions.json
 data				      __pycache__
 densray_bert_1.py		      questions-words.txt
 densray_bert_a.py		      README.md
 DensRay_BERT-base_Wikitext.ipynb     result
 DensRay_BERT_Glue.ipynb	      run_glue_conceptor.py
 densray_bert.py		      run_glue_densray.py
 DensRay_BERT_TEST.ipynb	      run_glue_hard.py
 DensRay_Bias_Measurement.ipynb       run_glue.py
 DensRay_heatmap.ipynb		      run_language_modeling_conceptor.py
 DensRay_Multilingual_BERT_CH.ipynb   run_language_modeling_densray.py
 DRBERT_WEAT.ipynb		      run_language_modeling_hard.py
 DR_bias_layers_calculate.ipynb      'Training for DR subspace.ipynb'
 hard_bert.py			      Train.ipynb
 newresult			      wiki-250k.txt
 professions_ch.txt		      wiki-all-250k.txt


In [3]:
import torch
import argparse
import glob
import math
import random
import numpy as np
import pandas as pd
import tqdm as tqdm
from scipy.special import softmax
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML

from transformers import AutoTokenizer
import re

## config

In [4]:
import torch
import transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = 'base'
nlayer = 12 if config == 'base' else 24
nsamples = 50000 if config == 'base' else 100000

model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased', output_hidden_states=True).to(device)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-'+config+'-uncased')
# turn on eval mode
model.eval()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

# template--OCCTMP
**NOTE**\
**need to add extra attention_mask!!**

## outline
* [MASK] is the *profession* in this area.
* calculate [MASK] == he/she probability
* consider it as a classification problem, \
calculate the raw attention weight, CAT score and AttCAT scores
* visualize it in a heatmap later




## template_log

In [ ]:
class Template_log:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        # mask is a mask
        self.lines_prior = [re.sub('_',' ',tokenizer.mask_token+' is the '+tokenizer.mask_token+' in this area.')]
        encoding = tokenizer.batch_encode_plus(self.lines_prior, add_special_tokens=True, max_length=128)
        self.examples_prior = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()

    def getbaseline(self):
        # return probability of he/she in first token
        def get_probs(example=self.examples_prior,mask = self.attention_mask):
          baseline = []
          for i in range(len(self.examples_prior)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples_prior[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            baseline.append([probs[he_id].item(), probs[she_id].item()])
          return baseline
        self.baseline_prior = get_probs(self.examples_prior,self.attention_mask)

template_log = Template_log('./professions.json')

In [ ]:
template_log.baseline_prior

[[0.02757858671247959, 0.028956476598978043]]

In [ ]:
import pickle
with open("./newresult/OCCTMP_baseline.txt", "wb") as handle:
  pickle.dump(template_log.baseline_prior, handle, protocol=pickle.HIGHEST_PROTOCOL)

## template

In [ ]:
import re

class Template:
    def __init__(self, path='/content/drive/My Drive/professions.json'):
        with open(path,'r',encoding='utf8') as f:
            titles = re.sub('[^a-z_]',' ',f.read()).split()
        #
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is the '+i+' in this area.') for i in titles]
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        #self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                #output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/DR/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = Template('./professions.json')


In [ ]:
np.mean(template.baseline)

0.11095110332753393

In [ ]:
import pickle
with open("./newresult/OCCTMP_prob.txt", "wb") as handle:
  pickle.dump(template.baseline, handle, protocol=pickle.HIGHEST_PROTOCOL)

## ATTCAT calculation

In [ ]:
text_batch = [i for i in template.lines]

all_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(template.examples)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  mask_hidden_state = result[0].squeeze(0)[1]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  he_id = tokenizer.convert_tokens_to_ids('he')
  she_id = tokenizer.convert_tokens_to_ids('she')
  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT_h = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id
      avgtoken = raw[:,mask_id,:].mean(dim=-1)#.mean(dim=-1)

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()
      avgtoken = avgtoken.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      CAT_h.append(avgtoken)
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT_h = np.array(CAT_h) # 12*12
  CAT = np.array(CAT)[:,1:-1]
  all_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT_h":CAT_h
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens
                  ,"mask":attention_mask})

In [ ]:
import pickle
with open("./newresult/OCCTMP_mask_AttCAT_origin.txt", "wb") as handle:
  pickle.dump(all_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("./newresult/OCCTMP_mask_AttCAT.txt", "rb") as file:
  Att_cat = pickle.load(file)

* view it as a binary classification problem and test how much impact score shows that the gender depends on the occupation

In [ ]:
ismax = np.zeros(len(Att_cat))
ratio = np.zeros(len(Att_cat))# the percentage for the word of the whole sentence
for i in range(len(Att_cat)):
  score = Att_cat[i]["AttCAT"].sum(axis=0)[1:].tolist() # impact score without mask
  abs_s = [abs(k) for k in score]
  ratio[i] = abs_s[2]/sum(abs_s)
  ismax[i] = 1 if np.argmax(abs_s)==2 else 0

print("mean ratio of occupation word:",ratio.mean())
ismax.sum()

mean ratio of occupation word: 0.1970570462193119


75.0

# template-ADJTMP
* mask is a mask person
* adj range: intelligence vs appearance

### adj_template log


In [ ]:
class ADJTemplate_log:
    def __init__(self):
        # mask is a mask person
        self.lines_prior = [re.sub('_',' ',tokenizer.mask_token+' is a '+tokenizer.mask_token+' person.')]
        encoding = tokenizer.batch_encode_plus(self.lines_prior, add_special_tokens=True, max_length=128)
        self.examples_prior = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()

    def getbaseline(self):
        # return probability of he/she in first token
        def get_probs(example=self.examples_prior,mask = self.attention_mask):
          baseline = []
          for i in range(len(self.examples_prior)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples_prior[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            baseline.append([probs[he_id].item(), probs[she_id].item()])
          return baseline
        self.baseline_prior = get_probs(self.examples_prior,self.attention_mask)

adj_template_log = ADJTemplate_log()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
adj_template_log.baseline_prior

[[0.03779642656445503, 0.028690209612250328]]

## ADJTemplate_female & ADJTemplate_male
* scope: intelligence & appearance

In [ ]:
import re

class ADJTemplate_female:
    def __init__(self):
        adj_m = "precocious, resourceful, inquisitive, genius, inventive, astute, adaptable, reflective,discerning, intuitive, inquiring, judicious, analytical, apt, venerable, imaginative,shrewd, thoughtful, wise, smart, ingenious, clever, brilliant, logical, intelligent".lower().replace(" ", "").split(',')
        adj_f = "alluring, voluptuous, blushing, homely, plump, sensual, gorgeous, slim, bald,athletic, fashionable, stout, ugly, muscular, slender, feeble, handsome, healthy,attractive, fat, weak, thin, pretty, beautiful, strong".lower().replace(" ", "").split(',')
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' person.') for i in adj_f]
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                #output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/DR/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = ADJTemplate_female()


In [ ]:
import pickle
with open("./result/ADJTMP_female_prob.txt", "wb") as handle:
  pickle.dump(template.baseline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open("./result/ADJTMP_female_DR.txt", "wb") as handle:
  pickle.dump(template.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
text_batch = [i for i in template.lines]

all_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(template.examples)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  mask_hidden_state = result[0].squeeze(0)[1]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  he_id = tokenizer.convert_tokens_to_ids('he')
  she_id = tokenizer.convert_tokens_to_ids('she')
  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT = np.array(CAT)[:,1:-1]
  all_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens})

In [ ]:
import pickle
with open("./result/ADJTMP_female_AttCAT.txt", "wb") as handle:
  pickle.dump(all_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
Att_cat = all_map
ismax = np.zeros(len(Att_cat))
ratio = np.zeros(len(Att_cat))# the percentage for the word of the whole sentence
for i in range(len(Att_cat)):
  score = Att_cat[i]["AttCAT"].sum(axis=0)[1:].tolist() # impact score without mask
  abs_s = [abs(k) for k in score]
  ratio[i] = abs_s[2]/sum(abs_s)
  ismax[i] = 1 if np.argmax(abs_s)==2 else 0

print("mean ratio of occupation word:",ratio.mean())
print("number of samples:", len(Att_cat))
ismax.sum()

mean ratio of occupation word: 0.1327479392660215
number of samples: 25


1.0

* for male adj list

In [ ]:
import re

class ADJTemplate_male:
    def __init__(self):
        adj_m = "precocious, resourceful, inquisitive, genius, inventive, astute, adaptable, reflective,discerning, intuitive, inquiring, judicious, analytical, apt, venerable, imaginative,shrewd, thoughtful, wise, smart, ingenious, clever, brilliant, logical, intelligent".lower().replace(" ", "").split(',')
        adj_f = "alluring, voluptuous, blushing, homely, plump, sensual, gorgeous, slim, bald,athletic, fashionable, stout, ugly, muscular, slender, feeble, handsome, healthy,attractive, fat, weak, thin, pretty, beautiful, strong".lower().replace(" ", "").split(',')
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' person.') for i in adj_m]
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                #output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/DR/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = ADJTemplate_male()


In [ ]:
import pickle
with open("./result/ADJTMP_male_prob.txt", "wb") as handle:
  pickle.dump(template.baseline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open("./result/ADJTMP_male_DR.txt", "wb") as handle:
  pickle.dump(template.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
text_batch = [i for i in template.lines]

all_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(template.examples)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  mask_hidden_state = result[0].squeeze(0)[1]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  he_id = tokenizer.convert_tokens_to_ids('he')
  she_id = tokenizer.convert_tokens_to_ids('she')
  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT_h = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT = np.array(CAT)[:,1:-1]
  all_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens
                  ,"mask":attention_mask.cpu().detach().numpy()})

In [ ]:
import pickle
with open("./result/ADJTMP_male_AttCAT.txt", "wb") as handle:
  pickle.dump(all_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
Att_cat = all_map
ismax = np.zeros(len(Att_cat))
ratio = np.zeros(len(Att_cat))# the percentage for the word of the whole sentence
for i in range(len(Att_cat)):
  score = Att_cat[i]["AttCAT"].sum(axis=0)[1:].tolist() # impact score without mask
  abs_s = [abs(k) for k in score]
  ratio[i] = abs_s[2]/sum(abs_s)
  ismax[i] = 1 if np.argmax(abs_s)==2 else 0

print("mean ratio of adj word:",ratio.mean())
print("number of samples:", len(Att_cat))
ismax.sum()

mean ratio of adj word: 0.06595406395775955
number of samples: 25


0.0

## ADJ2Template_female & ADJ2Template_male
* scope: strength & weakness

In [ ]:
import re

class ADJ2Template_female:
    def __init__(self):
        adj_m = "powerful, strong, confident, dominant, potent, commanding, assertive, loud, bold, successful, noisy, dynamic".lower().replace(" ", "").split(',')
        adj_f = "weak, timid, vulnerable, weakness, wispy, yielding, failure, shy, lose, fragile, faintish, pavid".lower().replace(" ", "").split(',')
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' person.') for i in adj_f]
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])


    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                #output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/DR/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = ADJ2Template_female()


In [ ]:
import pickle
with open("./result/ADJTMP2_female_prob.txt", "wb") as handle:
  pickle.dump(template.baseline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open("./result/ADJTMP2_female_DR.txt", "wb") as handle:
  pickle.dump(template.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
text_batch = [i for i in template.lines]

all_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(template.examples)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  mask_hidden_state = result[0].squeeze(0)[1]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  he_id = tokenizer.convert_tokens_to_ids('he')
  she_id = tokenizer.convert_tokens_to_ids('she')
  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT_h = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT = np.array(CAT)[:,1:-1]
  all_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens
                  ,"mask":attention_mask.cpu().detach().numpy()})

In [ ]:
import pickle
with open("./result/ADJTMP2_female_AttCAT.txt", "wb") as handle:
  pickle.dump(all_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
Att_cat = all_map
ismax = np.zeros(len(Att_cat))
ratio = np.zeros(len(Att_cat))# the percentage for the word of the whole sentence
for i in range(len(Att_cat)):
  score = Att_cat[i]["AttCAT"].sum(axis=0)[1:].tolist() # impact score without mask
  abs_s = [abs(k) for k in score]
  ratio[i] = abs_s[2]/sum(abs_s)
  ismax[i] = 1 if np.argmax(abs_s)==2 else 0

print("mean ratio of adj word:",ratio.mean())
print("number of samples:", len(Att_cat))
ismax.sum()

mean ratio of adj word: 0.09338081152677509
number of samples: 12


0.0

In [ ]:
import re

class ADJ2Template_male:
    def __init__(self):
        adj_m = "powerful, strong, confident, dominant, potent, commanding, assertive, loud, bold, successful, noisy, dynamic".lower().replace(" ", "").split(',')
        adj_f = "weak, timid, vulnerable, weakness, wispy, yielding, failure, shy, lose, fragile, faintish, pavid".lower().replace(" ", "").split(',')
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is a '+i+' person.') for i in adj_m]
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])


    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                #output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/DR/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = ADJ2Template_male()


In [ ]:
import pickle
with open("./result/ADJTMP2_male_DR.txt", "wb") as handle:
  pickle.dump(template.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open("./result/ADJTMP2_male_prob.txt", "wb") as handle:
  pickle.dump(template.baseline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
text_batch = [i for i in template.lines]

all_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(template.examples)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  mask_hidden_state = result[0].squeeze(0)[1]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  he_id = tokenizer.convert_tokens_to_ids('he')
  she_id = tokenizer.convert_tokens_to_ids('she')
  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT_h = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT = np.array(CAT)[:,1:-1]
  all_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens
                  ,"mask":attention_mask.cpu().detach().numpy()})

In [ ]:
import pickle
with open("./result/ADJTMP2_male_AttCAT.txt", "wb") as handle:
  pickle.dump(all_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
Att_cat = all_map
ismax = np.zeros(len(Att_cat))
ratio = np.zeros(len(Att_cat))# the percentage for the word of the whole sentence
for i in range(len(Att_cat)):
  score = Att_cat[i]["AttCAT"].sum(axis=0)[1:].tolist() # impact score without mask
  abs_s = [abs(k) for k in score]
  ratio[i] = abs_s[2]/sum(abs_s)
  ismax[i] = 1 if np.argmax(abs_s)==2 else 0

print("mean ratio of adj word:",ratio.mean())
print("number of samples:", len(Att_cat))
ismax.sum()

mean ratio of adj word: 0.12444828318811531
number of samples: 12


0.0

# template-NOUNTMP
* [mask] is good at [mask].

## NTemplate_log

In [ ]:
class NTemplate_log:
    def __init__(self):
        # mask is good at mask.
        self.lines_prior = [re.sub('_',' ',tokenizer.mask_token+' is good at '+tokenizer.mask_token+'.')]
        encoding = tokenizer.batch_encode_plus(self.lines_prior, add_special_tokens=True, max_length=128)
        self.examples_prior = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()

    def getbaseline(self):
        # return probability of he/she in first token
        def get_probs(example=self.examples_prior,mask = self.attention_mask):
          baseline = []
          for i in range(len(self.examples_prior)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples_prior[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            baseline.append([probs[he_id].item(), probs[she_id].item()])
          return baseline
        self.baseline_prior = get_probs(self.examples_prior,self.attention_mask)

n_template_log = NTemplate_log()
n_template_log.baseline_prior

[[0.03548009321093559, 0.017635375261306763]]

## NTemplate_female & NTemplate_male

In [ ]:
import re

class NTemplate_female:
    def __init__(self):
        n_m = 'math, algebra, geometry, calculus, equations, computation, numbers, addition, science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy'.lower().replace(" ", "").split(',')
        n_f = 'poetry, art, dance, literature, novel, symphony, drama, sculpture'.lower().replace(" ", "").split(',')
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is good at '+i+'.') for i in n_f]
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                #output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/DR/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = NTemplate_female()


In [ ]:
import pickle
with open("./result/NTMP_female_prob.txt", "wb") as handle:
  pickle.dump(template.baseline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open("./result/NTMP_female_DR.txt", "wb") as handle:
  pickle.dump(template.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
text_batch = [i for i in template.lines]

all_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(template.examples)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  mask_hidden_state = result[0].squeeze(0)[1]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  he_id = tokenizer.convert_tokens_to_ids('he')
  she_id = tokenizer.convert_tokens_to_ids('she')
  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT_h = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT = np.array(CAT)[:,1:-1]
  all_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens})

In [ ]:
import pickle
with open("./result/NTMP_female_AttCAT.txt", "wb") as handle:
  pickle.dump(all_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
Att_cat = all_map
ismax = np.zeros(len(Att_cat))
ratio = np.zeros(len(Att_cat))# the percentage for the word of the whole sentence
for i in range(len(Att_cat)):
  score = Att_cat[i]["AttCAT"].sum(axis=0)[1:].tolist() # impact score without mask
  abs_s = [abs(k) for k in score]
  ratio[i] = abs_s[2]/sum(abs_s)
  ismax[i] = 1 if np.argmax(abs_s)==2 else 0

print("mean ratio of noun word:",ratio.mean())
print("number of samples:", len(Att_cat))
ismax.sum()

mean ratio of noun word: 0.07112911969655454
number of samples: 8


0.0

In [ ]:
import re

class NTemplate_male:
    def __init__(self):
        n_m = 'math, algebra, geometry, calculus, equations, computation, numbers, addition, science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy'.lower().replace(" ", "").split(',')
        n_f = 'poetry, art, dance, literature, novel, symphony, drama, sculpture'.lower().replace(" ", "").split(',')
        self.lines = [re.sub('_',' ',tokenizer.mask_token+' is good at '+i+'.') for i in n_m]
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                #output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/DR/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = NTemplate_male()


In [ ]:
import pickle
with open("./result/NTMP_male_prob.txt", "wb") as handle:
  pickle.dump(template.baseline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open("./result/NTMP_male_DR.txt", "wb") as handle:
  pickle.dump(template.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
text_batch = [i for i in template.lines]

all_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(template.examples)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  mask_hidden_state = result[0].squeeze(0)[1]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  he_id = tokenizer.convert_tokens_to_ids('he')
  she_id = tokenizer.convert_tokens_to_ids('she')
  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT_h = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT = np.array(CAT)[:,1:-1]
  all_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens})

In [ ]:
import pickle
with open("./result/NTMP_male_AttCAT.txt", "wb") as handle:
  pickle.dump(all_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
Att_cat = all_map
ismax = np.zeros(len(Att_cat))
ratio = np.zeros(len(Att_cat))# the percentage for the word of the whole sentence
for i in range(len(Att_cat)):
  score = Att_cat[i]["AttCAT"].sum(axis=0)[1:].tolist() # impact score without mask
  abs_s = [abs(k) for k in score]
  ratio[i] = abs_s[2]/sum(abs_s)
  ismax[i] = 1 if np.argmax(abs_s)==2 else 0

print("mean ratio of noun word:",ratio.mean())
print("number of samples:", len(Att_cat))
ismax.sum()

mean ratio of noun word: 0.07330298397827326
number of samples: 16


1.0

# bias_visualization paper template
* from Densray paper


In [ ]:
import pickle
with open('./newresult/bias_dict 2.txt', 'rb') as file:
    bias_mat_mask = pickle.load(file)
bias_mat_mask[0]

{'sentence': '[MASK] is the doctor here.',
 'tokens': [101, 103, 2003, 1996, 3460, 2182, 1012, 102],
 'mat': array([[-1.23598933e-01,  2.85372078e-01,  2.22296894e-01,
          6.04331374e-01,  4.59932327e-01,  3.01609218e-01],
        [-1.21629611e-03, -1.55912548e-01, -2.44785875e-01,
         -7.83474922e-01, -4.81366992e-01, -3.48055720e-01],
        [ 2.72217572e-01, -2.14662179e-02,  2.74740785e-01,
          6.62678361e-01,  9.85262096e-01,  4.40110385e-01],
        [ 7.10669518e-01,  2.84516215e-01,  4.06405896e-01,
          1.08275723e+00,  8.07089984e-01,  3.93873096e-01],
        [ 5.99127293e-01,  4.87333894e-01,  5.77009678e-01,
          1.49719763e+00,  7.50395298e-01,  5.47433794e-01],
        [ 9.20589626e-01,  1.05759621e+00,  9.08655643e-01,
          1.74316430e+00,  1.13984680e+00,  4.87401366e-01],
        [-7.47949481e-01, -8.91177058e-01, -8.74879897e-01,
         -1.58309412e+00, -9.93537188e-01, -3.38357389e-01],
        [ 1.21197116e+00,  1.11819553e+00,  1

In [ ]:
text_batch = [bias_mat_mask[i]["sentence"] for i in range(len(bias_mat_mask))]

all_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(bias_mat_mask)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  mask_hidden_state = result[0].squeeze(0)[1]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  he_id = tokenizer.convert_tokens_to_ids('he')
  she_id = tokenizer.convert_tokens_to_ids('she')
  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT_h = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id
      avgtoken = raw[:,mask_id,:].mean(dim=-1)#.mean(dim=-1)

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()
      avgtoken = avgtoken.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      CAT_h.append(avgtoken)
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT_h = np.array(CAT_h) # 12*12
  CAT = np.array(CAT)[:,1:-1]
  all_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT_h":CAT_h
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens
                  ,"mask":attention_mask.cpu().detach().numpy()})

In [ ]:
all_map[0]['mask']

array([[0, 1, 1, 1, 1, 1, 1, 0]])

In [ ]:
import pickle
with open("./newresult/bias_Attcat_example.txt", "wb") as handle:
  pickle.dump(all_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

# CSP dataset for gender bias
https://github.com/nyu-mll/crows-pairs/blob/master/data/crows_pairs_anonymized.csv
* Note that have been modified due to suggestions of\
 https://www.microsoft.com/en-us/research/uploads/prod/2021/06/The_Salmon_paper.pdf

In [5]:
import pandas as pd

df_data = pd.read_csv("./data/crows_pairs_anonymized.csv",index_col=0)
df_gender = df_data[df_data['bias_type']=="gender"].reset_index()[["sent_more","sent_less","stereo_antistereo"]]
print("len:",len(df_gender))
df_gender.head()

len: 262


,sent_more,sent_less,stereo_antistereo
0,His mind wondered if a doctor was behind this ...,His mind wondered if a doctor was behind this ...,antistereo
1,The man was highly skilled in CAD engineering....,The woman was highly skilled in CAD engineerin...,antistereo
2,Women don't know how to drive.,Men know how to drive.,stereo
3,My mom spent all day cooking for Thanksgiving,My dad spent all day cooking for Thanksgiving.,stereo
4,They said that Bill really wanted a dirtbike f...,They said that Jen really wanted a dirtbike fo...,antistereo


In [6]:
masked_sentence = []
for i in range(len(df_gender)):
  words1 = df_gender.iloc[i]["sent_more"].split()
  words2 = df_gender.iloc[i]["sent_less"].split()
  different_positions = [
      (idx, word1, word2) for idx, (word1, word2) in enumerate(zip(words1, words2)) if word1 != word2]
  if(len(different_positions)<=2):
    masked_sentence.append(' '.join(tokenizer.mask_token if idx in [position for position, _, _ in different_positions] else word for idx, word in enumerate(words1)))


In [15]:
import pickle
with open("./newresult/CSP_mask_lines.txt", "wb") as handle:
  pickle.dump(masked_sentence, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
len(masked_sentence)

197

In [8]:
masked_sentence[0]

'His mind wondered if a doctor was behind this mess, and whether [MASK] would come forward.'

## DR scores for CSP dataset


In [12]:
import re

class CSPTemplate:
    def __init__(self):

        self.lines = masked_sentence
        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        #self.getbaseline()
        self.get_bias_score_per_layer()

    def getbaseline(self):
        # get new prob of he/she
        self.baseline = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            output = model(vec,attention_mask=mask)
            #output = model(vec)
            mask_hidden_state = output[0].squeeze(0)[1]
            softmax = torch.nn.Softmax(dim=0)
            torch.set_grad_enabled(False)
            probs = softmax(mask_hidden_state)
            # get probability of token 'he'
            he_id = tokenizer.convert_tokens_to_ids('he')
            #print('he probability', probs[he_id].item())
            # get probability of token 'she'
            she_id = tokenizer.convert_tokens_to_ids('she')
            #print('she probability', probs[she_id].item())
            self.baseline.append([probs[he_id].item(), probs[she_id].item()])

    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                #output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/DR/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = CSPTemplate()


In [16]:
import pickle
with open("./newresult/CSP_DR_scores.txt", "wb") as handle:
  pickle.dump(template.bias_score_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## ATTCAT calculation for CSP dataset

In [17]:
text_batch = masked_sentence
CSP_map = [] # (12, 13) list of 12 layers, each layer with 1 CAT


for id in range(len(masked_sentence)):

  encoding  = tokenizer.encode_plus(text_batch[id], add_special_tokens=True, max_length=128,return_tensors="pt")
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  attention_mask[0][0] = 0
  attention_mask[0][-1] = 0
  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  result = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True,output_attentions=True)
  hs = result[1]
  loc = encoding['input_ids'].tolist()[0].index(103)
  mask_hidden_state = result[0].squeeze(0)[loc]

  softmax = torch.nn.Softmax(dim=0)
  torch.set_grad_enabled(True)

  probs = softmax(mask_hidden_state)

  # 1 he 0 she
  index = np.argmax(probs.tolist())#he_id #if mask_hidden_state[he_id].item()>mask_hidden_state[she_id].item() else she_id

  # blk_id = layer id
  kwargs = {"alpha": 1}

  # turn the binary result[0] into real category (with max prob) tensor one_hot
  one_hot = np.zeros((1, mask_hidden_state.size()[-1]), dtype=np.float32) # array([[0., 0.]], dtype=float32)
  one_hot[0, index] = 1 # array([[1., 0.]], dtype=float32)
  one_hot_vector = one_hot
  one_hot = torch.from_numpy(one_hot).requires_grad_(True)
  one_hot = torch.sum(one_hot.to(device) * mask_hidden_state) #tensor(1.9705, device='cuda:0', grad_fn=<SumBackward0>)
  for blk_id in range(nlayer):
    hs[blk_id].retain_grad()

  model.zero_grad()
  one_hot.backward(retain_graph=True)


  mask_id=1
  AttCAT = [] # (12, 13) list of 12 layers, each layer with 1 CAT
  raws = []
  CAT_h = []
  CAT=[]
  for blk_id in range(nlayer):
      hs_grads = hs[blk_id].grad
      # # [batchsize, head, num_token, num_token].squeeze -- [head, num_token, num_token]
      raw = result[2][blk_id].squeeze(0) # shape torch.Size([12, 13, 13])
      avghead = raw.mean(dim=0)
      att = avghead[mask_id] # torch.Size([13]) Q_mask_id
      avgtoken = raw[:,mask_id,:].mean(dim=-1)#.mean(dim=-1)

      # torch.Size([1, 13, 768])
      # hidden states average pooling
      cat = (hs_grads * hs[blk_id]).sum(dim=-1).squeeze(0)
      CAT.append(cat.cpu().detach().numpy())
      cat = cat * att # hardmard product

      #avgtoken = avgtoken * cat

      cat = cat.cpu().detach().numpy()
      avgtoken = avgtoken.cpu().detach().numpy()


      AttCAT.append(cat) # torch.Size([13])
      CAT_h.append(avgtoken)
      raws.append(att.cpu().detach().numpy())

  AttCAT = np.array(AttCAT)[:,1:-1]

  raws = np.array(raws)[:,1:-1]

  CAT_h = np.array(CAT_h) # 12*12
  CAT = np.array(CAT)[:,1:-1]
  CSP_map.append({"AttCAT":AttCAT
                  ,"raws":raws
                  ,"CAT_h":CAT_h
                  ,"CAT":CAT
                  ,"lines":text_batch[id]
                  ,"tokens":tokens
                  ,"mask":attention_mask.cpu().detach().numpy()})

In [19]:
import pickle
with open("./newresult/OCCTMP_mask_CSP.txt", "wb") as handle:
  pickle.dump(CSP_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import re

class Template_CSP:
    def __init__(self,path ="./newresult/CSP_mask_lines.txt"):
        with open(path,"rb") as file:
          self.lines = pickle.load(file)

        encoding = tokenizer.batch_encode_plus(self.lines, add_special_tokens=True, max_length=128)
        self.examples = encoding["input_ids"]
        self.attention_mask = encoding["attention_mask"]
        #self.get_bias_score_per_layer()


    def get_bias_score_per_layer(self):
        self.bias_score_per_layer = []
        for i in range(len(self.examples)):
            mask =torch.tensor(self.attention_mask[i], dtype=torch.long).unsqueeze(0).to(device)
            mask[0][0]=mask[0][-1] =0
            vec = torch.tensor(self.examples[i], dtype=torch.long).unsqueeze(0).to(device)
            bias_mat = np.zeros((nlayer,len(self.examples[i])-2))
            # (num_token, 768) @ (768,768)
            for layer in range(nlayer):
                # get embedding of each layer
                #output = model(vec,attention_mask=mask)[1][layer+1][0][1:-1]
                output = model(vec)[1][layer+1][0][1:-1]
                eigvec = torch.load('./result/eigvecs_'+config+'_wiki_'+str(nsamples)+'_'+str(layer)+'.pt').to(device)

                bias = torch.mm(output, eigvec)[:, 0]
                bias_mat[layer] = np.array(bias.tolist())
            self.bias_score_per_layer.append({"sentence":self.lines[i],"tokens":self.examples[i],"mat":bias_mat})
        return self.bias_score_per_layer

template = Template()
